<a href="https://colab.research.google.com/github/rahul111jakhad/new/blob/main/validation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
%matplotlib inline
from datetime import timedelta
import tensorflow as tf
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_log_error
from datetime import datetime
import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
df_prev = pd.read_csv("http://www.exlanalytics.in/eq/frontend/web/casedata/EQ_2021_Data_Sample.csv" )
df_new =pd.read_csv("http://www.exlanalytics.in/eq/frontend/web/casedata/EQ-2021-Validation-Data-Sample.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
df_prev.rename(columns = {'Electricity_Sales_ Revenue_Residential ($ x1000)': 'Electricity_Sales_Revenue_Residential',
 'Electricity_Sales_ Sales_Residential (MWHr)': 'Electricity_Sales_Sales_Residential',
 'Electricity_Sales_ Customers_Count_Residential': 'Electricity_Sales_Customers_Count_Residential',
 'Electricity_Sales_ Revenue_Commercial ($ x1000)': 'Electricity_Sales_Revenue_Commercial',
 'Electricity_Sales_ Sales_Commercial (MWHr)': 'Electricity_Sales_Sales_Commercial',
 'Electricity_Sales_ Customers_Count_Commercial': 'Electricity_Sales_Customers_Count_Commercial',
 'Electricity_Sales_ Revenue_Industrial ($ x1000)': 'Electricity_Sales_Revenue_Industrial',
 'Electricity_Sales_ Sales_Industrial (MWHr)': 'Electricity_Sales_Sales_Industrial',
 'Electricity_Sales_ Customers_Count_Industrial': 'Electricity_Sales_Customers_Count_Industrial',
 'Electricity_Sales_ Revenue_Transportation ($ x1000)': 'Electricity_Sales_Revenue_Transportation',
 'Electricity_Sales_ Sales_Transportation (MWHr)': 'Electricity_Sales_Sales_Transportation',
 'Electricity_Sales_ Customers_Count_Transportation':'Electricity_Sales_Customers_Count_Transportation'}, inplace = True)

In [72]:
df_new.rename(columns = {'Electricity_Sales_Revenue_Residential($ x1000)': 'Electricity_Sales_Revenue_Residential',
 'Electricity_Sales_Sales_Residential(MWHr)' : 'Electricity_Sales_Sales_Residential',

 'Electricity_Sales_Revenue_Commercial($ x1000)': 'Electricity_Sales_Revenue_Commercial',
 'Electricity_Sales_Sales_Commercial(MWHr)': 'Electricity_Sales_Sales_Commercial',
 
 'Electricity_Sales_Revenue_Industrial($ x1000)': 'Electricity_Sales_Revenue_Industrial',
 'Electricity_Sales_Sales_Industrial(MWHr)': 'Electricity_Sales_Sales_Industrial' ,
 'Electricity_Sales_Revenue_Transportation ($ x1000)':'Electricity_Sales_Revenue_Transportation',
 
 'Electricity_Sales_Sales_Transportation(MWHr)': 'Electricity_Sales_Sales_Transportation'}, inplace = True)

In [73]:
df_new["confirmed_cases"] = 0
df_new["deaths"] = 0
df = pd.concat([df_prev , df_new])
df.date = pd.to_datetime(df.date , format = "%Y-%m-%d")
df = df.query("date>='"+"2021-01-11"+"' and date<='"+"2021-02-15"+"'")
df.sort_values(by=["stateFIPS","countyFIPS"] , inplace =True)
drop_col = ['google_mobility_parks' ,'apple_mobility_transit',"apple_mobility_walking","YoY_Reopened_Seated_Diner_Data"]
df.drop(columns = drop_col ,axis =1 , inplace = True)
for i in ['google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential', 'apple_mobility_driving',
       'Electricity_Sales_Revenue_Residential',
       'Electricity_Sales_Sales_Residential',
       'Electricity_Sales_Customers_Count_Residential',
       'Electricity_Sales_Revenue_Commercial',
       'Electricity_Sales_Sales_Commercial',
       'Electricity_Sales_Customers_Count_Commercial',
       'Electricity_Sales_Revenue_Industrial',
       'Electricity_Sales_Sales_Industrial',
       'Electricity_Sales_Customers_Count_Industrial',
       'Electricity_Sales_Revenue_Transportation',
       'Electricity_Sales_Sales_Transportation',
       'Electricity_Sales_Customers_Count_Transportation','DOMESTIC_Air_Passengers', 'INTERNATIONAL_Air_Passengers',
       'DOMESTIC_Flights_Cnt', 'INTERNATIONAL_Flights_Cnt','C_PCTPOVALL_2019',
       'C_PCTPOV017_2019', 'C_PCTPOV517_2019', 'C_MEDHHINC_2019', 'C_TOT_POP',
       'C_TOT_MALE', 'C_TOT_FEMALE', 'C_WA_MALE', 'C_WA_FEMALE',
       'C_MinorityPCT', 'C_BlackPCT', 'C_HispanicPCT', 'C_M_Labour_Force_corr',
       'C_M_Employed_corr', 'C_M_Unemployment_Rate_corr', 'S_D_dly_new_test',
       'S_D_cummulative_test', 'initclaims_count_regular_cw',
       'initclaims_rate_regular_cw', 'spend_all_cd', 'merchants_all_cd',
       'revenue_all_cd', 'critical_staffing_shortage_today_yes_SD',
       'critical_staffing_shortage_anticipated_within_week_yes_SD',
       'hospital_onset_covid_SD', 'hospital_onset_covid_coverage_SD',
       'inpatient_beds_SD', 'inpatient_beds_coverage_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_adult_covid_suspected_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'staffed_adult_icu_bed_occupancy_coverage_SD',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_SD',
       'staffed_icu_adult_patients_confirmed_covid_SD',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_adult_patients_hospitalized_confirmed_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD',
       'inpatient_beds_utilization_numerator_SD',
       'inpatient_beds_utilization_denominator_SD',
       'percent_of_inpatients_with_covid_SD',
       'inpatient_bed_covid_utilization_SD',
       'adult_icu_bed_covid_utilization_SD', 'adult_icu_bed_utilization_SD']:
    df.loc[df[i].isna() , i] = df.loc[df[i].notnull() , i].mean()
    

In [74]:
df["google_mobility"] = df["google_mobility_retail_and_recreation"] + df.google_mobility_grocery_and_pharmacy + df.google_mobility_transit_stations + df.google_mobility_workplaces  + df.google_mobility_residential 
df["electricity_sales"] = df["Electricity_Sales_Sales_Commercial"] +df["Electricity_Sales_Sales_Transportation"] + df["Electricity_Sales_Sales_Industrial"]
df["electricity_revenue"] = df["Electricity_Sales_Revenue_Residential"] + df["Electricity_Sales_Revenue_Industrial"] + df["Electricity_Sales_Revenue_Transportation"]
df["electricity_customers"] = df["Electricity_Sales_Customers_Count_Residential"] + df["Electricity_Sales_Customers_Count_Commercial"] + df["Electricity_Sales_Customers_Count_Transportation"]

df["air_passenger"] = df.DOMESTIC_Air_Passengers + df.INTERNATIONAL_Air_Passengers
df["total_flight"]  = df.DOMESTIC_Flights_Cnt + df.INTERNATIONAL_Flights_Cnt
df["onset_covid_inpatient_per_hospital"] = df['hospital_onset_covid_SD'] / df['hospital_onset_covid_coverage_SD']
df["inpatient_beds_per_hospital"] = df['inpatient_beds_SD'] / df['inpatient_beds_coverage_SD']
df["ICU_beds_occupancy_per_hospital"] = df['staffed_adult_icu_bed_occupancy_SD'] / df['staffed_adult_icu_bed_occupancy_coverage_SD']


In [75]:
len(df.columns)

87

In [76]:
new_df= df.copy()

In [77]:
new_df= new_df.drop(['google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy',
       'google_mobility_transit_stations', 'google_mobility_workplaces',
       'google_mobility_residential' ,
       'Electricity_Sales_Revenue_Commercial',
       'Electricity_Sales_Sales_Commercial',
       'Electricity_Sales_Customers_Count_Commercial',
       'Electricity_Sales_Revenue_Industrial',
       'Electricity_Sales_Sales_Industrial',
       'Electricity_Sales_Customers_Count_Industrial',
       'Electricity_Sales_Revenue_Transportation',
       'Electricity_Sales_Sales_Transportation',
       'Electricity_Sales_Customers_Count_Transportation' ,
       'DOMESTIC_Air_Passengers', 'INTERNATIONAL_Air_Passengers',
       'DOMESTIC_Flights_Cnt', 'INTERNATIONAL_Flights_Cnt', 'test_rate', 'new_test_rate', 
       'staffed_icu_adult_patients_confirmed_covid_SD',
              "hospital_onset_covid_SD",
              "hospital_onset_covid_coverage_SD",
              "inpatient_beds_SD",
              "inpatient_beds_coverage_SD",
              "staffed_adult_icu_bed_occupancy_SD",
              
              
        "total_adult_patients_hospitalized_confirmed_covid_SD",
        "total_pediatric_patients_hospitalized_confirmed_covid_SD",
             "staffed_adult_icu_bed_occupancy_coverage_SD",
        #'S_D_dly_new_test',
       'S_D_cummulative_test', 
              "initclaims_count_regular_cw"
              ] , axis =1)

In [78]:
max_value_df = pd.read_csv("max_value_traindata (1).csv" )
min_value_df = pd.read_csv("min_value_traindata (1).csv")

In [79]:
for feature_name in tqdm(new_df.columns):
    if(feature_name == "countyFIPS" or feature_name == "stateFIPS" or feature_name == "date" or feature_name == "confirmed_cases" or feature_name == "deaths"):
        continue
    max_value = max_value_df.loc[max_value_df.Attribute==feature_name , "max_value" ].values
    min_value = min_value_df.loc[min_value_df.Attribute==feature_name , "min_value" ].values
    new_df[feature_name] = (new_df[feature_name] - min_value) / (max_value - min_value)
    #new_df[feature_name] = (new_df[feature_name] ) / (max_value)

100%|██████████| 56/56 [00:00<00:00, 330.79it/s]


In [80]:
new_df.air_passenger = new_df.air_passenger.fillna(0)
new_df.total_flight = new_df.total_flight.fillna(0)

In [81]:
new_df.isna().sum().to_dict()

{'C_BlackPCT': 0,
 'C_HispanicPCT': 0,
 'C_MEDHHINC_2019': 0,
 'C_M_Employed_corr': 0,
 'C_M_Labour_Force_corr': 0,
 'C_M_Unemployment_Rate_corr': 0,
 'C_MinorityPCT': 0,
 'C_PCTPOV017_2019': 0,
 'C_PCTPOV517_2019': 0,
 'C_PCTPOVALL_2019': 0,
 'C_TOT_FEMALE': 0,
 'C_TOT_MALE': 0,
 'C_TOT_POP': 0,
 'C_WA_FEMALE': 0,
 'C_WA_MALE': 0,
 'Electricity_Sales_Customers_Count_Residential': 0,
 'Electricity_Sales_Revenue_Residential': 0,
 'Electricity_Sales_Sales_Residential': 0,
 'ICU_beds_occupancy_per_hospital': 0,
 'S_D_dly_new_test': 0,
 'adult_icu_bed_covid_utilization_SD': 0,
 'adult_icu_bed_utilization_SD': 0,
 'air_passenger': 0,
 'apple_mobility_driving': 0,
 'confirmed_cases': 0,
 'countyFIPS': 0,
 'critical_staffing_shortage_anticipated_within_week_yes_SD': 0,
 'critical_staffing_shortage_today_yes_SD': 0,
 'date': 0,
 'deaths': 0,
 'electricity_customers': 0,
 'electricity_revenue': 0,
 'electricity_sales': 0,
 'google_mobility': 0,
 'initclaims_rate_regular_cw': 0,
 'inpatient_bed_

In [82]:
sequence_length = 20

In [83]:
new_df.shape

(110030, 56)

In [84]:
import tensorflow as tf 
classifierLoad = tf.keras.models.load_model('new_final (2).h5')

In [89]:
#Will retrieve the number of cases and fatalities for the past 6 days from the given date
def build_inputs_for_date(country, province, date, df):
    start_date = date - timedelta(days=21)
    end_date = date - timedelta(days=1)
    str_start_date = start_date.strftime("%Y-%m-%d")
    str_end_date = end_date.strftime("%Y-%m-%d")
    df = df.query("stateFIPS=='"+str(country)+"' and countyFIPS=='"+str(province)+"' and date>='"+str_start_date+"' and date<='"+str_end_date+"'")
    #print(len(df.date) , df.date , province , country)
    #preparing the temporal inputs
    temporal_input_data = np.transpose(np.reshape(np.asarray([df.confirmed_cases ,
    df.apple_mobility_driving ,
    df.Electricity_Sales_Revenue_Residential ,
    df.Electricity_Sales_Sales_Residential ,
    df.Electricity_Sales_Customers_Count_Residential ,
    df.test_count ,
    df.new_test_count ,
    df.C_M_Labour_Force_corr ,
    df.C_M_Employed_corr ,
    df.C_M_Unemployment_Rate_corr ,
    df.S_D_dly_new_test ,
    df.initclaims_rate_regular_cw ,
    df.spend_all_cd ,
    df.critical_staffing_shortage_today_yes_SD ,
    df.critical_staffing_shortage_anticipated_within_week_yes_SD ,
    df.previous_day_admission_adult_covid_confirmed_SD ,
    df.previous_day_admission_adult_covid_suspected_SD ,
    df.previous_day_admission_pediatric_covid_confirmed_SD ,
    df.previous_day_admission_pediatric_covid_suspected_SD ,
    df.staffed_icu_adult_patients_confirmed_and_suspected_covid_SD ,
    df.total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD ,
    df.total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD ,
    df.inpatient_beds_utilization_SD ,
    df.inpatient_beds_utilization_numerator_SD ,
    df.inpatient_beds_utilization_denominator_SD ,
    df.percent_of_inpatients_with_covid_SD ,
    df.inpatient_bed_covid_utilization_SD ,
    df.adult_icu_bed_covid_utilization_SD ,
    df.adult_icu_bed_utilization_SD ,
    df.google_mobility ,
    df.electricity_sales ,
    df.electricity_revenue ,
    df.electricity_customers ,
    df.air_passenger ,
    df.total_flight ,
    df.onset_covid_inpatient_per_hospital ,
    df.inpatient_beds_per_hospital ,
    df.ICU_beds_occupancy_per_hospital,
    df.merchants_all_cd,
    df.revenue_all_cd

    ]),(40,sequence_length)), (1,0) ).astype(np.float32)
    
    #preparing all the demographic inputs
    C_PCTPOVALL_2019 = float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_PCTPOVALL_2019"].values)
    C_PCTPOV017_2019= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_PCTPOV017_2019"].values)
    C_PCTPOV517_2019= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_PCTPOV517_2019"].values)
    C_MEDHHINC_2019= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_MEDHHINC_2019"].values)
    C_TOT_POP= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_TOT_POP"].values)
    C_TOT_MALE =float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_TOT_MALE"].values)
    C_TOT_FEMALE = float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_TOT_FEMALE"].values)
    C_WA_MALE= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_WA_MALE"].values)
    C_WA_FEMALE= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_WA_FEMALE"].values)
    C_MinorityPCT= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_MinorityPCT"].values) 
    C_BlackPCT= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_BlackPCT"].values)
    C_HispanicPCT=float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "C_HispanicPCT"].values)
    #merchants_all_cd= float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "merchants_all_cd"].values)
    #revenue_all_cd = float(df.loc[((df.countyFIPS == province) & (df.stateFIPS == country)& (df.date==start_date))  , "revenue_all_cd"].values)
    
    demographic_input_data=[
                        C_PCTPOVALL_2019,
                       C_PCTPOV017_2019,
                       C_PCTPOV517_2019,
                       C_MEDHHINC_2019,
                       C_TOT_POP,C_TOT_MALE,
                       C_TOT_FEMALE,C_WA_MALE,
                       C_WA_FEMALE,
                       C_MinorityPCT,
                       C_BlackPCT,C_HispanicPCT,
                        ]
    
    return [np.array([temporal_input_data]), np.array([demographic_input_data])]

In [93]:
#Take a dataframe in input, will do the predictions and return the dataframe with extra rows
#containing the predictions
def predict_for_region(country, province, df):
    begin_prediction = "2021-02-01"
    start_date = datetime.strptime(begin_prediction,"%Y-%m-%d")
    end_prediction = "2021-02-15"
    end_date = datetime.strptime(end_prediction,"%Y-%m-%d")
    
    date_list = [start_date + timedelta(days=x) for x in range((end_date-start_date).days+1)]
   
    for date in date_list:
        input_data = build_inputs_for_date(country, province, date, df)
        result = classifierLoad.predict(input_data)
        
        #just ensuring that the outputs is
        #higher than the previous counts
        #result[0] = np.round(result[0])
        #if result[0]<input_data[0][0][-1][0]:
         #   result[0]=np.array([[input_data[0][0][-1][0]]])
        
        #result[1] = np.round(result[1])
        #if result[1]<input_data[0][0][-1][1]:
         #   result[1]=np.array([[input_data[0][0][-1][1]]])
        
        #We assign the quarantine and school status 
        #depending on previous values
        #e.g Once a country is locked, it will stay locked until the end
        df.loc[((df.countyFIPS == province) & (df.stateFIPS == country) & (df.date == date) ), "confirmed_cases"] = round(result[0][0])
        df.loc[((df.countyFIPS == province) & (df.stateFIPS == country) & (df.date == date) ), "model_predicted"] = round(result[0][0])
        print(country , province , round(result[0][0]) , date )
       
    return df

In [94]:
copy_df = new_df
copy_df["model_predicted"] = 0

In [95]:
#The functions that are called here need to optimise, sorry about that!

with tqdm(total=len(list(df_new.stateFIPS.unique()))) as pbar:
    for country in df_new.stateFIPS.unique():
        for province in df_new.query("stateFIPS=='"+str(country)+"'").countyFIPS.unique():
            copy_df = predict_for_region(country, province, copy_df )
             
        pbar.update(1)

  0%|          | 0/51 [00:00<?, ?it/s]

1 1001 5668 2021-02-01 00:00:00
1 1001 5715 2021-02-02 00:00:00
1 1001 5761 2021-02-03 00:00:00
1 1001 5805 2021-02-04 00:00:00
1 1001 5853 2021-02-05 00:00:00
1 1001 5901 2021-02-06 00:00:00
1 1001 5953 2021-02-07 00:00:00
1 1001 6007 2021-02-08 00:00:00
1 1001 6062 2021-02-09 00:00:00
1 1001 6117 2021-02-10 00:00:00
1 1001 6172 2021-02-11 00:00:00
1 1001 6230 2021-02-12 00:00:00
1 1001 6288 2021-02-13 00:00:00
1 1001 6349 2021-02-14 00:00:00
1 1001 6412 2021-02-15 00:00:00
1 1003 19056 2021-02-01 00:00:00
1 1003 19282 2021-02-02 00:00:00
1 1003 19509 2021-02-03 00:00:00
1 1003 19740 2021-02-04 00:00:00
1 1003 19976 2021-02-05 00:00:00
1 1003 20215 2021-02-06 00:00:00
1 1003 20460 2021-02-07 00:00:00
1 1003 20707 2021-02-08 00:00:00
1 1003 20953 2021-02-09 00:00:00
1 1003 21194 2021-02-10 00:00:00
1 1003 21427 2021-02-11 00:00:00
1 1003 21651 2021-02-12 00:00:00
1 1003 21865 2021-02-13 00:00:00
1 1003 22072 2021-02-14 00:00:00
1 1003 22272 2021-02-15 00:00:00
1 1005 1954 2021-02-01 00

  2%|▏         | 1/51 [01:15<1:03:07, 75.76s/it]

1 1133 2518 2021-02-14 00:00:00
1 1133 2529 2021-02-15 00:00:00
2 2013 44 2021-02-01 00:00:00
2 2013 46 2021-02-02 00:00:00
2 2013 49 2021-02-03 00:00:00
2 2013 52 2021-02-04 00:00:00
2 2013 55 2021-02-05 00:00:00
2 2013 58 2021-02-06 00:00:00
2 2013 61 2021-02-07 00:00:00
2 2013 64 2021-02-08 00:00:00
2 2013 67 2021-02-09 00:00:00
2 2013 71 2021-02-10 00:00:00
2 2013 75 2021-02-11 00:00:00
2 2013 79 2021-02-12 00:00:00
2 2013 83 2021-02-13 00:00:00
2 2013 88 2021-02-14 00:00:00
2 2013 93 2021-02-15 00:00:00
2 2016 151 2021-02-01 00:00:00
2 2016 159 2021-02-02 00:00:00
2 2016 167 2021-02-03 00:00:00
2 2016 175 2021-02-04 00:00:00
2 2016 183 2021-02-05 00:00:00
2 2016 192 2021-02-06 00:00:00
2 2016 200 2021-02-07 00:00:00
2 2016 209 2021-02-08 00:00:00
2 2016 219 2021-02-09 00:00:00
2 2016 229 2021-02-10 00:00:00
2 2016 240 2021-02-11 00:00:00
2 2016 251 2021-02-12 00:00:00
2 2016 263 2021-02-13 00:00:00
2 2016 277 2021-02-14 00:00:00
2 2016 291 2021-02-15 00:00:00
2 2020 24504 2021-02-

  4%|▍         | 2/51 [01:48<51:24, 62.94s/it]  

...

54 54109 1871 2021-02-15 00:00:00
55 55001 1684 2021-02-01 00:00:00
55 55001 1697 2021-02-02 00:00:00
55 55001 1710 2021-02-03 00:00:00
55 55001 1721 2021-02-04 00:00:00
55 55001 1735 2021-02-05 00:00:00
55 55001 1748 2021-02-06 00:00:00
55 55001 1762 2021-02-07 00:00:00
55 55001 1777 2021-02-08 00:00:00
55 55001 1793 2021-02-09 00:00:00
55 55001 1808 2021-02-10 00:00:00
55 55001 1821 2021-02-11 00:00:00
55 55001 1836 2021-02-12 00:00:00
55 55001 1850 2021-02-13 00:00:00
55 55001 1867 2021-02-14 00:00:00
55 55001 1884 2021-02-15 00:00:00
55 55003 1218 2021-02-01 00:00:00
55 55003 1226 2021-02-02 00:00:00
55 55003 1232 2021-02-03 00:00:00
55 55003 1238 2021-02-04 00:00:00
55 55003 1245 2021-02-05 00:00:00
55 55003 1253 2021-02-06 00:00:00
55 55003 1260 2021-02-07 00:00:00
55 55003 1269 2021-02-08 00:00:00
55 55003 1278 2021-02-09 00:00:00
55 55003 1287 2021-02-10 00:00:00
55 55003 1295 2021-02-11 00:00:00
55 55003 1303 2021-02-12 00:00:00
55 55003 1312 2021-02-13 00:00:00
55 55003 1322 

 98%|█████████▊| 50/51 [1:00:30<01:22, 82.61s/it]

55 55141 8052 2021-02-13 00:00:00
55 55141 8128 2021-02-14 00:00:00
55 55141 8204 2021-02-15 00:00:00
56 56001 3918 2021-02-01 00:00:00
56 56001 3954 2021-02-02 00:00:00
56 56001 3991 2021-02-03 00:00:00
56 56001 4026 2021-02-04 00:00:00
56 56001 4061 2021-02-05 00:00:00
56 56001 4099 2021-02-06 00:00:00
56 56001 4133 2021-02-07 00:00:00
56 56001 4167 2021-02-08 00:00:00
56 56001 4205 2021-02-09 00:00:00
56 56001 4245 2021-02-10 00:00:00
56 56001 4283 2021-02-11 00:00:00
56 56001 4322 2021-02-12 00:00:00
56 56001 4362 2021-02-13 00:00:00
56 56001 4403 2021-02-14 00:00:00
56 56001 4442 2021-02-15 00:00:00
56 56003 994 2021-02-01 00:00:00
56 56003 1013 2021-02-02 00:00:00
56 56003 1033 2021-02-03 00:00:00
56 56003 1049 2021-02-04 00:00:00
56 56003 1067 2021-02-05 00:00:00
56 56003 1085 2021-02-06 00:00:00
56 56003 1104 2021-02-07 00:00:00
56 56003 1122 2021-02-08 00:00:00
56 56003 1143 2021-02-09 00:00:00
56 56003 1164 2021-02-10 00:00:00
56 56003 1184 2021-02-11 00:00:00
56 56003 1205 2

100%|██████████| 51/51 [1:00:57<00:00, 71.72s/it]

56 56045 788 2021-02-14 00:00:00
56 56045 805 2021-02-15 00:00:00


In [96]:
cc = copy_df.copy()

In [97]:
cc = copy_df.query("date>='"+"2021-02-01"+"' and date<='"+"2021-02-15"+"'")

In [98]:
final = cc.loc[: , ["stateFIPS" , "countyFIPS" , "date" , "model_predicted"]]

In [99]:
final.to_excel("after_NNoob_180103059_180104019_IITGUWAHATI.xlsx")